In [48]:
from constants import *

In [49]:
def predict_with_onnxruntime(model_def, *inputs):
    import onnxruntime as ort

    sess = ort.InferenceSession(
        model_def.SerializeToString(), providers=["CPUExecutionProvider"]
    )
    names = [i.name for i in sess.get_inputs()]
    dinputs = {name: input for name, input in zip(names, inputs)}
    res = sess.run(None, dinputs)
    names = [o.name for o in sess.get_outputs()]
    return {name: output for name, output in zip(names, res)}


In [50]:
import torch
torch.manual_seed(0)
example_x = torch.randn(3, 40)

*Example of usage ONNX operator. More can be found here: https://github.com/onnx/onnx/blob/main/docs/Operators.md*

In [54]:
from onnx import helper
from onnx import TensorProto

mean_1_onx = helper.make_node('ReduceMean', ['X'], ['Y'], axes=[0])

mean_1_graph = helper.make_graph(
  [mean_1_onx], 
  'mean_1', 
  [helper.make_tensor_value_info('X', TensorProto.FLOAT, [None, 40])],
  [helper.make_tensor_value_info('Y', TensorProto.FLOAT, [None, 40])]
  )
original_model = helper.make_model(mean_1_graph, producer_name='onnx-mean', opset_imports=[helper.make_opsetid("", 13)])

Y = predict_with_onnxruntime(original_model, example_x.numpy())
print(Y)

with open(f"{EXPORTED_MODELS_PATH}/mean_1.onnx", "wb") as f:
    f.write(original_model.SerializeToString())

{'Y': array([[-4.40499276e-01,  1.09044671e-01, -4.59520549e-01,
        -1.26826495e-01,  7.37821639e-01,  8.64726782e-01,
        -2.48481557e-01, -1.11458504e+00, -1.98938653e-01,
         6.37337625e-01, -1.58782840e-01,  1.45416871e-01,
        -3.85157555e-01, -3.68657708e-02,  5.64534605e-01,
         2.25503054e-02, -5.10013103e-02, -7.70693123e-01,
         8.10731888e-01,  1.57462633e+00, -1.18188918e+00,
        -1.49523485e+00,  4.16707724e-01,  5.27144134e-01,
        -1.73527405e-01,  4.57053542e-01,  1.11249745e-01,
         6.48622513e-02,  1.38080365e-03,  1.16248918e+00,
         3.60408574e-01, -4.12715286e-01, -2.89290339e-01,
         4.71600480e-02, -1.65318042e-01, -4.22524780e-01,
        -1.48972511e-01,  4.45227534e-01, -8.49007741e-02,
         4.97076124e-01]], dtype=float32)}


*Sklearn substitution:*

In [56]:
from skl2onnx.algebra.onnx_ops import OnnxReduceMean_13

mean_onnx = OnnxReduceMean_13('X', axes=[0])
onnx_model = mean_onnx.to_onnx({'X': example_x.numpy()})

Y = predict_with_onnxruntime(onnx_model, example_x.numpy())
print(Y)
onnx_model

{'Re_reduced0': array([[-4.40499276e-01,  1.09044671e-01, -4.59520549e-01,
        -1.26826495e-01,  7.37821639e-01,  8.64726782e-01,
        -2.48481557e-01, -1.11458504e+00, -1.98938653e-01,
         6.37337625e-01, -1.58782840e-01,  1.45416871e-01,
        -3.85157555e-01, -3.68657708e-02,  5.64534605e-01,
         2.25503054e-02, -5.10013103e-02, -7.70693123e-01,
         8.10731888e-01,  1.57462633e+00, -1.18188918e+00,
        -1.49523485e+00,  4.16707724e-01,  5.27144134e-01,
        -1.73527405e-01,  4.57053542e-01,  1.11249745e-01,
         6.48622513e-02,  1.38080365e-03,  1.16248918e+00,
         3.60408574e-01, -4.12715286e-01, -2.89290339e-01,
         4.71600480e-02, -1.65318042e-01, -4.22524780e-01,
        -1.48972511e-01,  4.45227534e-01, -8.49007741e-02,
         4.97076124e-01]], dtype=float32)}


ir_version: 7
producer_name: "skl2onnx"
producer_version: "1.17.0"
domain: "ai.onnx"
model_version: 0
graph {
  node {
    input: "X"
    output: "Re_reduced0"
    name: "Re_ReduceMean"
    op_type: "ReduceMean"
    attribute {
      name: "axes"
      ints: 0
      type: INTS
    }
    domain: ""
  }
  name: "OnnxReduceMean_13"
  input {
    name: "X"
    type {
      tensor_type {
        elem_type: 1
        shape {
          dim {
          }
          dim {
            dim_value: 40
          }
        }
      }
    }
  }
  output {
    name: "Re_reduced0"
    type {
      tensor_type {
        elem_type: 1
        shape {
          dim {
            dim_value: 1
          }
          dim {
            dim_value: 40
          }
        }
      }
    }
  }
}
opset_import {
  domain: ""
  version: 13
}